# Hebden Rainfall Data

This workbook collates and visualises a set of rainfall data related to Hebden Bridge.


First of all, let's install some libraries.

In [1]:
!pip install --quiet folium

In [2]:
hebden_bridge = { 'lat': 53.762318, 'long': -2.019955 }

The data used comes from the [Environment Agency Flood Monitoring API][EA_FLOOD]    

In [3]:
import requests
import json

root="http://environment.data.gov.uk/flood-monitoring"

def download_station(filename, query):
    url="{root}/id/stations".format(root=root)
    r = requests.get(url, params={**query, "_limit": 50})
    print(r.url)
    with open(filename, 'w') as f:
        f.write(r.text)

def to_geojson(list):
    def emit_feature(i):
        return {
          "type": "Feature",
          "geometry": {
            "type": "Point",
            "coordinates": [round(i['long'], 4), round(i['lat'], 4)]
          },
          "properties": {
            "stationReference": i['stationReference'],
            "id": i['@id']
          }
        }
    res = {
        "type": "FeatureCollection",
        "features": [emit_feature(x) for x in list]
    }
    return res

def read_data(filename, downloader):
    try:
        f = open(filename, 'r')
    except FileNotFoundError:
        downloader(filename)
        f = open(filename, 'r')
    finally:
        data = json.loads(f.read())
        f.close()
    return data

get_rainfall_stations = lambda f: download_station(f,
                                                   {
                                                       **hebden_bridge,
                                                       "dist": 10,
                                                       "parameter": "rainfall"
                                                   }
                                                  )
get_level_stations =    lambda f: download_station(f,
                                                   {
                                                       **hebden_bridge,
                                                       "dist": 10,
                                                       "parameter": "level"
                                                   }
                                                  )


rainfall_stations = to_geojson(read_data('rainfall_stations.json',
                               downloader=get_rainfall_stations)['items'])
level_stations = to_geojson(read_data('level_stations.json',
                            downloader=get_level_stations)['items'])
# Output linted using http://geojsonlint.com/

Finally, we can visualise the location of the rainfall and river gauges on a map.

In [4]:
import folium

m = folium.Map(
    location=[hebden_bridge['lat'], hebden_bridge['long']],
    tiles='Stamen Terrain',
    zoom_start=12
)

station_tooltip = lambda: folium.GeoJsonTooltip(['stationReference', 'id'], ['Ref', 'ID'])

def my_popup(x):
    return "<p>Ref: " + x['properties']['stationReference'] + "<br>" + \
            "Id: " + x['properties']['id'] + "</p>"

rainfall_layer = folium.FeatureGroup(name='Rainfall Stations').add_to(m)
level_layer = folium.FeatureGroup(name='Level Stations').add_to(m)

# folium.GeoJson(rainfall_stations, name="Rainfall Stations", tooltip=station_tooltip()).add_to(m)
for s in rainfall_stations['features']:
    folium.Marker(s['geometry']['coordinates'][::-1],
                  icon=folium.Icon(color='blue', icon='cloud'),
                  popup=my_popup(s)
                 ).add_to(rainfall_layer)

# folium.GeoJson(level_stations, name="Level Stations", tooltip=station_tooltip(), style_function=level_style).add_to(m)
for s in level_stations['features']:
    folium.Marker(s['geometry']['coordinates'][::-1],
                  icon=folium.Icon(color='green', icon='list'),
                  popup=my_popup(s)
                 ).add_to(level_layer)

folium.LayerControl().add_to(m)

m.save('stations.html')

In [5]:
m